In [ ]:
import torch
import pandas as pd
from torch_geometric.nn import HeteroConv, Linear, SAGEConv, GATConv, to_hetero
import torch.nn.functional as F
import torch_geometric.transforms as T
from torch_geometric.profile import count_parameters



In [ ]:
class IdentityEncoder:
    def __init__(self, dtype=None):
        self.dtype = dtype

    def __call__(self, df):
        return torch.from_numpy(df.values).view(-1, 1).to(self.dtype)

In [ ]:
def load_node_csv(path, index_col, encoders=None, **kwargs):
    df = pd.read_csv(path, index_col=index_col, **kwargs)
    mapping = {index: i for i, index in enumerate(df.index.unique())}

    x = None
    if encoders is not None:
        xs = [encoder(df[col]) for col, encoder in encoders.items()]
        x = torch.cat(xs, dim=-1)

    return x, mapping

In [ ]:
def load_edge_csv(path, src_index_col, src_mapping, dst_index_col, dst_mapping,
                  attr_encoders=None,label_encoders = None ,**kwargs):
    df = pd.read_csv(path, **kwargs)

    src = [src_mapping[index] for index in df[src_index_col]]
    dst = [dst_mapping[index] for index in df[dst_index_col]]
    edge_index = torch.tensor([src, dst])

    edge_attr = None
    if attr_encoders is not None:
        edge_attrs = [encoder(df[col]) for col, encoder in attr_encoders.items()]
        edge_attr = torch.cat(edge_attrs, dim=-1)
    if label_encoders is not None:
        edge_labels = [encoder(df[col]) for col, encoder in label_encoders.items()]
        edge_label = torch.cat(edge_labels, dim=-1)

    return edge_index, edge_attr, edge_label

In [ ]:
customer_encoders = {col_name : IdentityEncoder(dtype=torch.float32) for col_name in ["n_orders","n_not_returned","n_partial_returned","n_fully_returned","n_products"]}
order_encoders = {col_name : IdentityEncoder(dtype=torch.float32) for col_name in ["label","n_products"]}
product_encoders = {col_name : IdentityEncoder(dtype=torch.float32) for col_name in ["n_orders", "n_returned"]}
customer_x, customer_mapping = load_node_csv('data/customer_node_attributes.csv', index_col='customer', encoders=customer_encoders)
order_x, order_mapping = load_node_csv('data/order_node_attributes.csv', index_col='order', encoders=order_encoders)
product_x, product_mapping = load_node_csv('data/product_node_attributes.csv', index_col='product', encoders=product_encoders)

In [ ]:
customer_order_edge_attr_encoders = {col_name : IdentityEncoder(dtype=torch.float32) for col_name in ["n_products","train_masks","valid_masks","test_masks"]}
customer_order_edge_label_encoders = {"label" : IdentityEncoder(torch.long)}
customer_order_edge_index, customer_order_edge_attr, customer_order_edge_label = load_edge_csv('data/customer_edgelist.csv', 'customer', customer_mapping, 'order', order_mapping, attr_encoders=customer_order_edge_attr_encoders, label_encoders=customer_order_edge_label_encoders)
order_product_edge_index, order_product_edge_attr, order_product_edge_label = load_edge_csv('data/order_edgelist.csv', 'order', order_mapping, 'product', product_mapping, label_encoders={"label": IdentityEncoder(torch.long)})

In [ ]:
from torch_geometric.data import HeteroData

data = HeteroData()

customer_mapping = []
order_mapping = []
item_mapping = []


data['order'].x = order_x
data['product'].x = product_x
data['customer'].x = customer_x
data['customer', 'order'].edge_index = customer_order_edge_index
data['customer', 'order'].edge_attr = customer_order_edge_attr
data['customer', 'order'].edge_label = customer_order_edge_label

data['order',  'product'].edge_index = order_product_edge_index
data['order',  'product'].edge_attr = order_product_edge_attr
data['order',  'product'].edge_label = order_product_edge_label

data = T.ToUndirected()(data)

print(data)

In [ ]:
data.has_isolated_nodes()

In [ ]:
class GAT(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GATConv((-1, -1), hidden_channels, add_self_loops=False)
        self.lin1 = Linear(-1, hidden_channels)
        self.conv2 = GATConv((-1, -1), out_channels, add_self_loops=False)
        self.lin2 = Linear(-1, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index) + self.lin1(x)
        x = x.relu()
        x = self.conv2(x, edge_index) + self.lin2(x)
        return x
    
class GNN(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = SAGEConv((-1, -1), hidden_channels)
        self.conv2 = SAGEConv((-1, -1), out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        return x

In [ ]:
# DOES NOT WORK FROM HERE
# model = GNN(hidden_channels=64, out_channels=3)
model = GAT(hidden_channels =  256,out_channels =3)
model = to_hetero(model, data.metadata(), aggr='sum')

In [ ]:
data.num_nodes

In [ ]:
count_parameters(model)

In [ ]:
model

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

def train():
    model.train()
    optimizer.zero_grad()
    output = model(data.x_dict, data.edge_index_dict,
                 )
    # edge_attr[:,-3] - train mask
    mask = data['customer', 'order'].edge_attr[:,-3].bool()
    pred = F.softmax(output["order"][mask], dim=-1)
    target = data['customer', 'order'].edge_label[mask].squeeze()
    print(output["order"][mask][0])
    loss = F.cross_entropy(pred, target)
    loss.backward()
    optimizer.step()
    return float(loss)

In [ ]:
for epoch in range(1, 15):
    loss = train()
    print(f'Epoch: {epoch:02d}, Loss: {loss:.4f}')


In [ ]:
# Example of target with class indices
input = torch.randn(3, 5, requires_grad=True)
target = torch.randint(5, (3,), dtype=torch.int64)
loss = F.cross_entropy(input, target)

print(input.shape, target.shape, loss)
# Example of target with class probabilities
# input = torch.randn(3, 5, requires_grad=True)
# target = torch.randn(3, 5).softmax(dim=1)
# loss = F.cross_entropy(input, target)


In [ ]:
data['customer', 'order'].edge_label.unique()
